<a href="https://colab.research.google.com/github/Dusein/MachineLearningTask/blob/main/10thWeekTask/Classification_model_MLP_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Ganti 'your_file_path' dengan path sebenarnya ke file CSV Anda di Google Drive
file_path = '/content/drive/My Drive/Dataset/ENB2012_data.xlsx'

# Membaca file CSV ke dalam DataFrame
data = pd.read_excel(file_path)

# Menampilkan beberapa baris pertama dari DataFrame
print(data.head())

     X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2
0  0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33
1  0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33
2  0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33
3  0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33
4  0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28


In [7]:
# Memisahkan fitur dan label
X = data.iloc[:, :-2].values
y = data.iloc[:, -2:].values

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Konversi ke Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Dataset dan DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [8]:
from torch import nn

# Definisi model MLP
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_function):
        super(MLPModel, self).__init__()
        layers = []
        in_features = input_size
        for out_features in hidden_layers:
            layers.append(nn.Linear(in_features, out_features))
            layers.append(activation_function)
            in_features = out_features
        layers.append(nn.Linear(in_features, y_train.shape[1]))  # Output layer
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


In [10]:
from torch import optim
import matplotlib.pyplot as plt

# Fungsi untuk melatih model
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    train_losses, test_losses = [], []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_losses.append(running_loss / len(train_loader))

        # Evaluasi
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
            test_losses.append(test_loss / len(test_loader))

    return train_losses, test_losses


In [11]:
# Bandingkan jumlah hidden layers
batch_size = 32
epochs = 50
activation_function = nn.ReLU()
learning_rate = 0.01

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

hidden_layer_results = {}
for num_layers in [1, 2, 3]:
    for num_neurons in [4, 8, 16, 32, 64]:
        hidden_layers = [num_neurons] * num_layers
        model = MLPModel(X_train.shape[1], hidden_layers, activation_function)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()
        train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, epochs)
        hidden_layer_results[(num_layers, num_neurons)] = test_losses[-1]

# Tampilkan hasil
hidden_layer_results


{(1, 4): 9.543527317047118,
 (1, 8): 9.764016723632812,
 (1, 16): 9.210935688018798,
 (1, 32): 8.821922969818115,
 (1, 64): 8.276722621917724,
 (2, 4): 9.492184066772461,
 (2, 8): 9.286012744903564,
 (2, 16): 8.990169525146484,
 (2, 32): 8.750315475463868,
 (2, 64): 3.3351603507995606,
 (3, 4): 9.521539211273193,
 (3, 8): 8.638978481292725,
 (3, 16): 7.915019798278808,
 (3, 32): 3.338445854187012,
 (3, 64): 4.904811000823974}

In [13]:
hidden_layer_df = pd.DataFrame.from_dict(hidden_layer_results, orient='index', columns=['Test Loss'])
hidden_layer_df.index = pd.MultiIndex.from_tuples(hidden_layer_df.index, names=['Hidden Layers', 'Neurons per Layer'])
hidden_layer_df = hidden_layer_df.reset_index()
hidden_layer_df.sort_values(by='Test Loss', inplace=True)
hidden_layer_df

,Hidden Layers,Neurons per Layer,Test Loss
9,2,64,3.335160
13,3,32,3.338446
14,3,64,4.904811
12,3,16,7.915020
4,1,64,8.276723
11,3,8,8.638978
8,2,32,8.750315
3,1,32,8.821923
7,2,16,8.990170
2,1,16,9.210936


In [14]:
activation_functions = {
    "Linear": nn.Identity(),
    "Sigmoid": nn.Sigmoid(),
    "ReLU": nn.ReLU(),
    "Softmax": nn.Softmax(dim=1),
    "Tanh": nn.Tanh()
}

activation_results = {}
hidden_layers = [16, 32]
for name, activation in activation_functions.items():
    model = MLPModel(X_train.shape[1], hidden_layers, activation)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, epochs)
    activation_results[name] = test_losses[-1]

# Tampilkan hasil
activation_function_df = pd.DataFrame(list(activation_results.items()), columns=['Activation Function', 'Test Loss'])
activation_function_df.sort_values(by='Test Loss', inplace=True)
activation_function_df


,Activation Function,Test Loss
4,Tanh,2.994936
2,ReLU,8.683222
1,Sigmoid,8.774968
0,Linear,9.576995
3,Softmax,141.379008


In [15]:
epoch_results = {}
for num_epochs in [1, 10, 25, 50, 100, 250]:
    model = MLPModel(X_train.shape[1], hidden_layers, activation_function)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs)
    epoch_results[num_epochs] = test_losses[-1]

# Tampilkan hasil
epoch_df = pd.DataFrame(list(epoch_results.items()), columns=['Epochs', 'Test Loss'])
epoch_df.sort_values(by='Test Loss', inplace=True)
epoch_df


,Epochs,Test Loss
5,250,1.633502
4,100,2.028990
2,25,9.227448
3,50,9.245501
1,10,12.280978
0,1,274.240891


In [16]:
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
lr_results = {}
for lr in learning_rates:
    model = MLPModel(X_train.shape[1], hidden_layers, activation_function)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, epochs)
    lr_results[lr] = test_losses[-1]

# Tampilkan hasil
learning_rate_df = pd.DataFrame(list(lr_results.items()), columns=['Learning Rate', 'Test Loss'])
learning_rate_df.sort_values(by='Test Loss', inplace=True)
learning_rate_df


,Learning Rate,Test Loss
2,0.1000,2.398042
3,0.0100,8.283093
4,0.0010,12.922218
1,1.0000,99.717502
5,0.0001,430.200372
0,10.0000,2331.715576


In [17]:
batch_sizes = [16, 32, 64, 128, 256, 512]
batch_results = {}
for batch_size in batch_sizes:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    model = MLPModel(X_train.shape[1], hidden_layers, activation_function)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    train_losses, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, epochs)
    batch_results[batch_size] = test_losses[-1]

# Tampilkan hasil
batch_size_df = pd.DataFrame(list(batch_results.items()), columns=['Batch Size', 'Test Loss'])
batch_size_df.sort_values(by='Test Loss', inplace=True)
batch_size_df


,Batch Size,Test Loss
0,16,2.981691
1,32,5.470445
2,64,8.581578
3,128,9.271353
4,256,12.094777
5,512,18.861059


In [19]:
best_results = {
    'Hyperparameter': ['Hidden Layers & Neurons', 'Activation Function', 'Epochs', 'Learning Rate', 'Batch Size'],
    'Best Value': [
        min(hidden_layer_results, key=hidden_layer_results.get),
        min(activation_results, key=activation_results.get),
        min(epoch_results, key=epoch_results.get),
        min(lr_results, key=lr_results.get),
        min(batch_results, key=batch_results.get)
    ],
    'Test Loss': [
        hidden_layer_results[min(hidden_layer_results, key=hidden_layer_results.get)],
        activation_results[min(activation_results, key=activation_results.get)],
        epoch_results[min(epoch_results, key=epoch_results.get)],
        lr_results[min(lr_results, key=lr_results.get)],
        batch_results[min(batch_results, key=batch_results.get)]
    ]
}

best_results_df = pd.DataFrame(best_results)
best_results_df


,Hyperparameter,Best Value,Test Loss
0,Hidden Layers & Neurons,"(2, 64)",3.335160
1,Activation Function,Tanh,2.994936
2,Epochs,250,1.633502
3,Learning Rate,0.1,2.398042
4,Batch Size,16,2.981691
